# Contents
## 01. Importing Libraries
## 02. Consistency Checks
## 03. Plot a choropleth - COVID Deaths dataframe
## 04. Plot a choropleth - Contributing Conditions dataframe

# 01. Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
# This command prompts matplotlib visuals to appear in the notebook 
%matplotlib inline

In [3]:
# import .json file
country_geo = r'/Users/kirstinhelgeson/Desktop/Data Analytics Course/A6/us-states.json'

In [4]:
# That's just in case you want to look at the JSON file contents here too:
f = open(r'/Users/kirstinhelgeson/Desktop/Data Analytics Course/A6/us-states.json',)

# returns JSON object as a dictionary
data = json.load(f)
  
# Iterating through the json list
for i in data['features']:
    print(i)

{'type': 'Feature', 'id': 'AL', 'properties': {'name': 'Alabama'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-87.359296, 35.00118], [-85.606675, 34.984749], [-85.431413, 34.124869], [-85.184951, 32.859696], [-85.069935, 32.580372], [-84.960397, 32.421541], [-85.004212, 32.322956], [-84.889196, 32.262709], [-85.058981, 32.13674], [-85.053504, 32.01077], [-85.141136, 31.840985], [-85.042551, 31.539753], [-85.113751, 31.27686], [-85.004212, 31.003013], [-85.497137, 30.997536], [-87.600282, 30.997536], [-87.633143, 30.86609], [-87.408589, 30.674397], [-87.446927, 30.510088], [-87.37025, 30.427934], [-87.518128, 30.280057], [-87.655051, 30.247195], [-87.90699, 30.411504], [-87.934375, 30.657966], [-88.011052, 30.685351], [-88.10416, 30.499135], [-88.137022, 30.318396], [-88.394438, 30.367688], [-88.471115, 31.895754], [-88.241084, 33.796253], [-88.098683, 34.891641], [-88.202745, 34.995703], [-87.359296, 35.00118]]]}}
{'type': 'Feature', 'id': 'AK', 'properties': {'name': 'Alaska'},

In [5]:
# create path to import data
path = r'/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project'

In [6]:
# import COVID deaths data
df_covid_deaths = pd.read_csv(os.path.join(path,'02 Data','Prepared Data','COVID_deaths_wrangled_cleaned.csv'), index_col = False)

In [7]:
# check output of COVID deaths data
df_covid_deaths.head()

,Year,Month,State,Age Group,Footnote,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths"
0,2020,1,Alabama,All Ages,One or more data cells have counts between 1-9...,4,4729,282,0,35,318
1,2020,1,Alabama,Under 1 year,NaN,0,31,0,0,0,0
2,2020,1,Alabama,1-4 years,One or more data cells have counts between 1-9...,0,12,4,0,0,4
3,2020,1,Alabama,5-14 years,One or more data cells have counts between 1-9...,0,16,4,0,0,4
4,2020,1,Alabama,15-24 years,One or more data cells have counts between 1-9...,0,56,0,0,4,4


In [8]:
# check shape of COVID deaths data
df_covid_deaths.shape

(26832, 11)

In [9]:
# import contributing conditions data
df_conditions = pd.read_csv(os.path.join(path,'02 Data','Prepared Data','Contributing_conditions_wrangled_cleaned.csv'), index_col = False)

In [10]:
# check output of contributing conditions data
df_conditions.head()

,Year,Month,State,Condition Group,Condition,Age Group,Flag,COVID-19 Deaths
0,2020,1,Alabama,Respiratory diseases,Influenza and pneumonia,0-24,NaN,0
1,2020,2,Alabama,Respiratory diseases,Influenza and pneumonia,0-24,NaN,0
2,2020,3,Alabama,Respiratory diseases,Influenza and pneumonia,0-24,NaN,0
3,2020,4,Alabama,Respiratory diseases,Influenza and pneumonia,0-24,NaN,0
4,2020,5,Alabama,Respiratory diseases,Influenza and pneumonia,0-24,NaN,0


In [11]:
# check shape of contributing conditions data
df_conditions.shape

(502320, 8)

# 02. Consistency Checks

In [12]:
# check for missing values in COVID deaths dataframe
df_covid_deaths.isnull().sum()

Year                                           0
Month                                          0
State                                          0
Age Group                                      0
Footnote                                    8165
COVID-19 Deaths                                0
Total Deaths                                   0
Pneumonia Deaths                               0
Pneumonia and COVID-19 Deaths                  0
Influenza Deaths                               0
Pneumonia, Influenza, or COVID-19 Deaths       0
dtype: int64

In [13]:
# check for missing values in contributing conditions dataframe
df_conditions.isnull().sum()

Year                    0
Month                   0
State                   0
Condition Group         0
Condition               0
Age Group               0
Flag               346118
COVID-19 Deaths         0
dtype: int64

In [14]:
# check for duplicates in COVID deaths dataframe
dups1 = df_covid_deaths.duplicated()

In [15]:
dups1.shape

(26832,)

### No duplicates found.

In [16]:
# check for duplicates in contributing conditions dataframe
dups2 = df_conditions.duplicated()

In [17]:
dups2.shape

(502320,)

### No duplicates found.

# 03. Plot a choropleth - COVID Deaths dataframe

In [18]:
# create a dataframe with just the states and the values we want plotted
    #filter out rows with "All Ages" in the "Age Group" column
filtered_df_1 = df_covid_deaths[df_covid_deaths['Age Group'] != 'All Ages']
    # group the data by state and calculate the total COVID-19 deaths for each state
data_to_plot1 = filtered_df_1.groupby('State')['COVID-19 Deaths'].sum().reset_index()
data_to_plot1.head()

,State,COVID-19 Deaths
0,Alabama,21372
1,Alaska,1620
2,Arizona,30230
3,Arkansas,12637
4,California,108326


In [20]:
folder_path = '/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/02 Data/Prepared Data'
output_file_path = os.path.join(folder_path, 'deaths_by_state20-23.csv')
data_to_plot1.to_csv(output_file_path, index=False)

## TOTAL COVID DEATHS 2020 - 2023 IN THE US

In [21]:
# Setup a folium map at a high-level zoom
map_1 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot1,
    columns = ['State', 'COVID-19 Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Purples', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "COVID-19 Deaths").add_to(map_1)
folium.LayerControl().add_to(map_1)

map_1

In [25]:
# save map as html file
map_1.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/covid19_map.html")

### The states with the most COVID-19 deaths from 2020 - 2023 are California, Texas, New York and Florida. Other southern states are shaded slightly darker but it is not guaranteed that if it's a southern state, it will have more COVID-19 deaths.

In [22]:
# create a dataframe with just the states and the values we want plotted
    #filter out rows with "All Ages" in the "Age Group" column
filtered_df_2 = df_covid_deaths[df_covid_deaths['Age Group'] != 'All Ages']
    # group the data by state and calculate the total pneumonia deaths for each state
data_to_plot2 = filtered_df_2.groupby('State')['Pneumonia Deaths'].sum().reset_index()
data_to_plot2.head()

,State,Pneumonia Deaths
0,Alabama,17210
1,Alaska,1546
2,Arizona,29804
3,Arkansas,13128
4,California,121551


## TOTAL COVID DEATHS 2020

In [23]:
# create a dataframe with just the states and the values we want plotted
    #filter for rows with "All Ages" in the "Age Group" column
filtered_df_2020 = df_covid_deaths[df_covid_deaths['Age Group'] == 'All Ages']
    # filter for rows with "2020" in the "Year" column
filtered_df_2020 = filtered_df_2020[filtered_df_2020['Year'] == 2020]
    # group the data by state and calculate the total COVID-19 deaths for each state
data_to_plot_2020 = filtered_df_2020.groupby('State')['COVID-19 Deaths'].sum().reset_index()
data_to_plot_2020.head()

,State,COVID-19 Deaths
0,Alabama,6712
1,Alaska,259
2,Arizona,9321
3,Arkansas,4027
4,California,33583


In [24]:
folder_path_2 = '/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/02 Data/Prepared Data'
output_file_path_2 = os.path.join(folder_path, 'deaths_by_state2020.csv')
data_to_plot_2020.to_csv(output_file_path_2, index=False)

In [25]:
# Setup a folium map at a high-level zoom
map_2020 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot_2020,
    columns = ['State', 'COVID-19 Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Purples', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "COVID-19 Deaths").add_to(map_2020)
folium.LayerControl().add_to(map_2020)

map_2020

In [25]:
# save map as html file
map_2020.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/covid19_2020_map.html")

## TOTAL COVID DEATHS 2021

In [26]:
# create a dataframe with just the states and the values we want plotted
    #filter for rows with "All Ages" in the "Age Group" column
filtered_df_2021 = df_covid_deaths[df_covid_deaths['Age Group'] == 'All Ages']
    # filter for rows with "2020" in the "Year" column
filtered_df_2021 = filtered_df_2021[filtered_df_2021['Year'] == 2021]
    # group the data by state and calculate the total COVID-19 deaths for each state
data_to_plot_2021 = filtered_df_2021.groupby('State')['COVID-19 Deaths'].sum().reset_index()
data_to_plot_2021.head()

,State,COVID-19 Deaths
0,Alabama,9719
1,Alaska,839
2,Arizona,14060
3,Arkansas,5336
4,California,48329


In [27]:
# Setup a folium map at a high-level zoom
map_2021 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot_2021,
    columns = ['State', 'COVID-19 Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Purples', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "COVID-19 Deaths").add_to(map_2021)
folium.LayerControl().add_to(map_2021)

map_2021

In [28]:
# save map as html file
map_2021.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/covid19_2021_map.html")

## TOTAL COVID DEATHS 2022

In [28]:
# create a dataframe with just the states and the values we want plotted
    #filter for rows with "All Ages" in the "Age Group" column
filtered_df_2022 = df_covid_deaths[df_covid_deaths['Age Group'] == 'All Ages']
    # filter for rows with "2020" in the "Year" column
filtered_df_2022 = filtered_df_2022[filtered_df_2022['Year'] == 2022]
    # group the data by state and calculate the total COVID-19 deaths for each state
data_to_plot_2022 = filtered_df_2022.groupby('State')['COVID-19 Deaths'].sum().reset_index()
data_to_plot_2022.head()

,State,COVID-19 Deaths
0,Alabama,4227
1,Alaska,322
2,Arizona,5906
3,Arkansas,2773
4,California,22413


In [29]:
# Setup a folium map at a high-level zoom
map_2022 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot_2022,
    columns = ['State', 'COVID-19 Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Purples', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "COVID-19 Deaths").add_to(map_2022)
folium.LayerControl().add_to(map_2022)

map_2022

In [31]:
# save map as html file
map_2022.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/covid19_2022_map.html")

## TOTAL COVID DEATHS 2023

In [30]:
# create a dataframe with just the states and the values we want plotted
    #filter for rows with "All Ages" in the "Age Group" column
filtered_df_2023 = df_covid_deaths[df_covid_deaths['Age Group'] == 'All Ages']
    # filter for rows with "2020" in the "Year" column
filtered_df_2023 = filtered_df_2023[filtered_df_2023['Year'] == 2023]
    # group the data by state and calculate the total COVID-19 deaths for each state
data_to_plot_2023 = filtered_df_2023.groupby('State')['COVID-19 Deaths'].sum().reset_index()
data_to_plot_2023.head()

,State,COVID-19 Deaths
0,Alabama,673
1,Alaska,33
2,Arizona,844
3,Arkansas,422
4,California,3875


In [31]:
folder_path = '/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/02 Data/Prepared Data'
output_file_path_3 = os.path.join(folder_path, 'deaths_by_state2023.csv')
data_to_plot_2023.to_csv(output_file_path_3, index=False)

In [33]:
# Setup a folium map at a high-level zoom
map_2023 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot_2023,
    columns = ['State', 'COVID-19 Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Purples', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "COVID-19 Deaths").add_to(map_2023)
folium.LayerControl().add_to(map_2023)

map_2023

In [34]:
# save map as html file
map_2023.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/covid19_2023_map.html")

## TOTAL PNEUMONIA DEATHS 2020 - 2023 IN THE US

In [27]:
# Setup a folium map at a high-level zoom
map_2 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot2,
    columns = ['State', 'Pneumonia Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Purples', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Pneumonia Deaths").add_to(map_2)
folium.LayerControl().add_to(map_2)

map_2

In [28]:
# save map as html file
map_2.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/pneumonia_map.html")

### The most pneumonia deaths from 2020 - 2023 occurred in California, Texas, Florida, and New York. Ohio and Pennsylvania also have a notable amount of pneumonia deaths.

## TOTAL INFLUENZA DEATHS 2020 - 2023 IN THE US

In [29]:
# create a dataframe with just the states and the values we want plotted
    #filter out rows with "All Ages" in the "Age Group" column
filtered_df_3 = df_covid_deaths[df_covid_deaths['Age Group'] != 'All Ages']
    # group the data by state and calculate the total influenza deaths for each state
data_to_plot3 = filtered_df_3.groupby('State')['Influenza Deaths'].sum().reset_index()
data_to_plot3.head()

,State,Influenza Deaths
0,Alabama,568
1,Alaska,144
2,Arizona,581
3,Arkansas,449
4,California,1995


In [30]:
# Setup a folium map at a high-level zoom
map_3 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot3,
    columns = ['State', 'Influenza Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Purples', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Influenza Deaths").add_to(map_3)
folium.LayerControl().add_to(map_3)

map_3

In [31]:
# save map as html file
map_3.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/influenza_map.html")

### It should be noted that the death counts for influenza are significantly lower than either COVID-19 or pneumonia. The most influenza deaths from 2020 - 2023 occurred in New York, California, Texas, and Florida.

# 04. Plot a choropleth - Contributing Conditions dataframe

## Respiratory Failure-related COVID-19 Deaths

In [32]:
# create a dataframe with just the states and the values we want plotted
    #filter out rows with "All Ages" in the "Age Group" column
filtered_df_4 = df_conditions[df_conditions['Age Group'] != 'All Ages']
    #filter for only one condition, 'respiratory failure'
filtered_df_4 = filtered_df_4[filtered_df_4['Condition'] == 'Respiratory failure']
    # group the data by state and calculate the total COVID-19 deaths for each state
data_to_plot4 = filtered_df_4.groupby('State')['COVID-19 Deaths'].sum().reset_index()
data_to_plot4.head()

,State,COVID-19 Deaths
0,Alabama,8637
1,Alaska,753
2,Arizona,13851
3,Arkansas,6048
4,California,52191


In [33]:
# Setup a folium map at a high-level zoom
map_4 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot4,
    columns = ['State', 'COVID-19 Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Greens', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Respiratory Failure-related COVID-19 Deaths").add_to(map_4)
folium.LayerControl().add_to(map_4)

map_4

In [34]:
# save map as html file
map_4.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/respiratory_failure_map.html")

### The most respiratory failure-related COVID-19 deaths from 2020 - 2023 occurred in California, Texas, Florida, New York, Pennsylvania and Ohio.

## Cardiac Arrest-related COVID-19 Deaths

In [35]:
# create a dataframe with just the states and the values we want plotted
    #filter out rows with "All Ages" in the "Age Group" column
filtered_df_5 = df_conditions[df_conditions['Age Group'] != 'All Ages']
    #filter for only one condition, 'cardiac arrest'
filtered_df_5 = filtered_df_5[filtered_df_5['Condition'] == 'Cardiac arrest']
    # group the data by state and calculate the total COVID-19 deaths for each state
data_to_plot5 = filtered_df_5.groupby('State')['COVID-19 Deaths'].sum().reset_index()
data_to_plot5.head()

,State,COVID-19 Deaths
0,Alabama,3593
1,Alaska,415
2,Arizona,3934
3,Arkansas,1854
4,California,29991


In [36]:
# Setup a folium map at a high-level zoom
map_5 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot5,
    columns = ['State', 'COVID-19 Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Greens', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Cardiac arrest-related COVID-19 Deaths").add_to(map_5)
folium.LayerControl().add_to(map_5)

map_5

In [37]:
# save map as html file
map_5.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/cardiac_arrest_map.html")

### The most cardiac arrest-related COVID-19 deaths occurred in California, New York, Texas, Georgia, Florida, and New Jersey.

## Diabetes-related COVID-19 Deaths

In [38]:
# create a dataframe with just the states and the values we want plotted
    #filter out rows with "All Ages" in the "Age Group" column
filtered_df_6 = df_conditions[df_conditions['Age Group'] != 'All Ages']
    #filter for only one condition, 'diabetes'
filtered_df_6 = filtered_df_6[filtered_df_6['Condition'] == 'Diabetes']
    # group the data by state and calculate the total COVID-19 deaths for each state
data_to_plot6 = filtered_df_6.groupby('State')['COVID-19 Deaths'].sum().reset_index()
data_to_plot6.head()

,State,COVID-19 Deaths
0,Alabama,2430
1,Alaska,325
2,Arizona,4283
3,Arkansas,1934
4,California,22243


In [39]:
# Setup a folium map at a high-level zoom
map_6 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot6,
    columns = ['State', 'COVID-19 Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Greens', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Diabetes-related COVID-19 Deaths").add_to(map_6)
folium.LayerControl().add_to(map_6)

map_6

In [40]:
# save map as html file
map_6.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/diabetes_map.html")

### the most diabetes-related COVID-19 deaths occurred in California, Texas, Florida, New York and Illinois.

## Hypertensive Diseases-related COVID-19 Deaths

In [41]:
# create a dataframe with just the states and the values we want plotted
    #filter out rows with "All Ages" in the "Age Group" column
filtered_df_7 = df_conditions[df_conditions['Age Group'] != 'All Ages']
    #filter for only one condition, 'hypertensive diseases'
filtered_df_7 = filtered_df_7[filtered_df_7['Condition'] == 'Hypertensive diseases']
    # group the data by state and calculate the total COVID-19 deaths for each state
data_to_plot7 = filtered_df_7.groupby('State')['COVID-19 Deaths'].sum().reset_index()
data_to_plot7.head()

,State,COVID-19 Deaths
0,Alabama,2450
1,Alaska,400
2,Arizona,4514
3,Arkansas,2451
4,California,22391


In [42]:
# Setup a folium map at a high-level zoom
map_7 = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot7,
    columns = ['State', 'COVID-19 Deaths'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'Greens', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Hypertensive diseases-related COVID-19 Deaths").add_to(map_7)
folium.LayerControl().add_to(map_7)

map_7

In [43]:
# save map as html file
map_7.save("/Users/kirstinhelgeson/Desktop/Data Analytics Course/07-2023 Achievement 6 Project/04 Analysis/Visualizations/hypertensive_map.html")

### The most hypertensive diseases-related COVID-19 deaths occurred in California, Texas, Florida, New York, and Illinois.

## The states most commonly mentioned here are California, Texas, Florida and New York. These states have high populations, so it might be worth making some maps that show proportion of state populations but being able to see the sheer amount of people affected by these illnesses is still useful. My hypothesis that southern states have more COVID-19 deaths is not true in all cases so I will discontinue analysis in that direction. No particular region is more likely to have more COVID-19 deaths.